# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,36.6177,-121.9166,15.24,87,0,1.54,US,1696302578
1,1,lata,40.1629,-8.3327,16.15,95,2,1.28,PT,1696302578
2,2,puerto natales,-51.7236,-72.4875,2.90,96,100,1.90,CL,1696302579
3,3,wailua homesteads,22.0669,-159.3780,27.24,79,40,3.60,US,1696302579
4,4,san rafael,-34.6177,-68.3301,15.62,30,97,4.60,AR,1696302579


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',  # Use humidity as color
    s='Humidity',  # Use humidity as size
    cmap='Blues',       # Specify the color map
    colorbar=True,      # Show the color bar
    title='City Humidity Map',
    width=800,
    height=600
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_df=city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']< 27) & (city_data_df['Wind Speed']< 4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
reduced_df.dropna()

# Display sample data
reduced_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,glenvale,-27.5648,151.9076,24.38,49,0,1.34,AU,1696302584
43,43,termas de rio hondo,-27.4998,-64.8604,23.67,42,0,3.02,AR,1696302584
98,98,saint-pierre,-21.3393,55.4781,21.98,68,0,2.57,RE,1696302596
121,121,anar darah,32.7587,61.6540,23.80,16,0,1.29,AF,1696302599
148,148,hamilton,39.1834,-84.5333,21.89,80,0,0.00,US,1696302420
185,185,paranhos,-23.8928,-55.4311,22.79,57,0,2.69,BR,1696302610
195,195,pamandzi,-12.7967,45.2794,25.88,83,0,3.60,YT,1696302611
209,209,highlands,27.3670,-81.3498,24.52,94,0,3.09,US,1696302582
223,223,laguna,38.4210,-121.4238,22.16,44,0,2.57,US,1696302617
228,228,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.54,GF,1696302617


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_df[["City","Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,glenvale,AU,-27.5648,151.9076,49,
43,termas de rio hondo,AR,-27.4998,-64.8604,42,
98,saint-pierre,RE,-21.3393,55.4781,68,
121,anar darah,AF,32.7587,61.6540,16,
148,hamilton,US,39.1834,-84.5333,80,
185,paranhos,BR,-23.8928,-55.4311,57,
195,pamandzi,YT,-12.7967,45.2794,83,
209,highlands,US,27.3670,-81.3498,94,
223,laguna,US,38.4210,-121.4238,44,
228,remire-montjoly,GF,4.9167,-52.2667,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 1000
limit = 20 

params = {
   
   "limit":limit,
  
   "apiKey":geoapify_key,
   "categories" :"accommodation.hotel"

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude =hotel_df.loc[index, "Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url , params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url , params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


glenvale - nearest hotel: No hotel found
termas de rio hondo - nearest hotel: Hotel Panamericano
saint-pierre - nearest hotel: Tropic Hotel
anar darah - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
paranhos - nearest hotel: No hotel found
pamandzi - nearest hotel: Gite Le Madina
highlands - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
ghat - nearest hotel: فندق تاسيلي
as sulayyil - nearest hotel: No hotel found
beau vallon - nearest hotel: Le Jardin de Beau Vallon
cangzhou - nearest hotel: No hotel found
sidi akkacha - nearest hotel: No hotel found
hanceville - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
40,glenvale,AU,-27.5648,151.9076,49,No hotel found
43,termas de rio hondo,AR,-27.4998,-64.8604,42,Hotel Panamericano
98,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
121,anar darah,AF,32.7587,61.6540,16,No hotel found
148,hamilton,US,39.1834,-84.5333,80,No hotel found
185,paranhos,BR,-23.8928,-55.4311,57,No hotel found
195,pamandzi,YT,-12.7967,45.2794,83,Gite Le Madina
209,highlands,US,27.3670,-81.3498,94,No hotel found
223,laguna,US,38.4210,-121.4238,44,No hotel found
228,remire-montjoly,GF,4.9167,-52.2667,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display



# Configure the map plot
# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    s='Humidity',
    cmap='Blues',
    colorbar=True,
    title='City Humidity Map',
    width=800,
    height=600,
    hover_cols=['Hotel Name', 'Country', 'Humidity (%)']  # Customize the hover tooltip
)
# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,Country)